# 🌱 CropGuard - CCMT Disease Detection Model Training

**Dataset:** CCMT (Cashew, Cassava, Maize, Tomato)  
**Model:** MobileNetV2 (Lightweight, ~14MB)  
**Classes:** 22 diseases across 4 crops

---

## 📊 Dataset Structure:
- **Cashew** (5 classes): anthracnose, gumosis, healthy, leaf miner, red rust
- **Cassava** (5 classes): bacterial blight, brown spot, green mite, healthy, mosaic
- **Maize** (7 classes): fall armyworm, grasshopper, healthy, leaf beetle, leaf blight, leaf spot, streak virus
- **Tomato** (5 classes): healthy, leaf blight, leaf curl, septoria leaf spot, verticillium wilt

**Total: 22 disease classes**

## 🔧 Step 1: Setup & Installations

In [ ]:
# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)
print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))

# Install additional packages if needed
!pip install -q pillow matplotlib seaborn scikit-learn

## 📂 Step 2: Upload Dataset to Colab

**Option A:** Upload ZIP file  
**Option B:** Mount Google Drive (recommended)

In [ ]:
# Option A: Upload dataset ZIP file
# Uncomment these lines if you uploaded a ZIP:
# !unzip -q "/content/CCMT_Dataset.zip" -d /content/dataset
# DATA_DIR = "/content/dataset/CCMT Dataset"

# Option B: Mount Google Drive (RECOMMENDED)
from google.colab import drive
drive.mount('/content/drive')

# Update this path to where your dataset is in Google Drive
DATA_DIR = "/content/drive/MyDrive/CCMT Dataset"  # <-- CHANGE THIS PATH

# Verify dataset exists
import os
print("\n📁 Dataset contents:")
!ls "{DATA_DIR}"

## 📊 Step 3: Explore Dataset

In [ ]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Count images per class
dataset_stats = {}
total_images = 0

for crop in os.listdir(DATA_DIR):
    crop_path = os.path.join(DATA_DIR, crop)
    if not os.path.isdir(crop_path):
        continue
    
    dataset_stats[crop] = {}
    for disease in os.listdir(crop_path):
        disease_path = os.path.join(crop_path, disease)
        if not os.path.isdir(disease_path):
            continue
        
        images = [f for f in os.listdir(disease_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        count = len(images)
        dataset_stats[crop][disease] = count
        total_images += count

# Display stats
print("\n📊 Dataset Statistics:\n")
for crop, diseases in dataset_stats.items():
    print(f"\n🌾 {crop.upper()}:")
    for disease, count in diseases.items():
        print(f"   {disease}: {count} images")
    print(f"   TOTAL: {sum(diseases.values())} images")

print(f"\n\n🎯 TOTAL IMAGES: {total_images}")
print(f"📦 Total Classes: {sum(len(d) for d in dataset_stats.values())}")

In [ ]:
# Visualize class distribution
plt.figure(figsize=(14, 6))

# Prepare data for plotting
all_classes = []
all_counts = []
colors_map = {'Cashew': '#FF6B6B', 'Cassava': '#4ECDC4', 'Maize': '#FFE66D', 'Tomato': '#FF4757'}
colors = []

for crop, diseases in dataset_stats.items():
    for disease, count in diseases.items():
        all_classes.append(f"{crop}_{disease}")
        all_counts.append(count)
        colors.append(colors_map.get(crop, '#95A5A6'))

plt.bar(range(len(all_classes)), all_counts, color=colors, alpha=0.8, edgecolor='black')
plt.xticks(range(len(all_classes)), all_classes, rotation=90, ha='right')
plt.xlabel('Disease Class', fontsize=12, fontweight='bold')
plt.ylabel('Number of Images', fontsize=12, fontweight='bold')
plt.title('🌱 CCMT Dataset - Class Distribution', fontsize=16, fontweight='bold')
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("\n✅ Dataset exploration complete!")

## 🖼️ Step 4: Visualize Sample Images

In [ ]:
# Display sample images from each crop
fig, axes = plt.subplots(4, 5, figsize=(18, 14))
fig.suptitle('🌱 Sample Images from CCMT Dataset', fontsize=18, fontweight='bold', y=0.995)

crop_idx = 0
for crop in ['Cashew', 'Cassava', 'Maize', 'Tomato']:
    crop_path = os.path.join(DATA_DIR, crop)
    diseases = [d for d in os.listdir(crop_path) if os.path.isdir(os.path.join(crop_path, d))]
    
    for col_idx in range(min(5, len(diseases))):
        disease = diseases[col_idx]
        disease_path = os.path.join(crop_path, disease)
        images = [f for f in os.listdir(disease_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        if images:
            img_path = os.path.join(disease_path, images[0])
            img = Image.open(img_path)
            axes[crop_idx, col_idx].imshow(img)
            axes[crop_idx, col_idx].set_title(f"{crop}\n{disease}", fontsize=10, fontweight='bold')
            axes[crop_idx, col_idx].axis('off')
    
    crop_idx += 1

plt.tight_layout()
plt.show()

## 🔨 Step 5: Prepare Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Image parameters
IMG_SIZE = 224
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

# Data augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=VALIDATION_SPLIT
)

# No augmentation for validation
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=VALIDATION_SPLIT
)

# Create generators
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Get class names
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)

print(f"\n✅ Data prepared!")
print(f"📊 Training samples: {train_generator.samples}")
print(f"📊 Validation samples: {validation_generator.samples}")
print(f"🎯 Number of classes: {num_classes}")
print(f"\n📝 Classes: {class_names}")

## 🧠 Step 6: Build Model

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained MobileNetV2 (without top layer)
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base model initially
base_model.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n🎯 Model built successfully!")
model.summary()

## 🚀 Step 7: Train Model (Phase 1 - Transfer Learning)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import time

# Callbacks
checkpoint = ModelCheckpoint(
    'ccmt_model_phase1.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=0.00001,
    verbose=1
)

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=8,
    restore_best_weights=True,
    verbose=1
)

# Train Phase 1
print("\n🚀 Starting Phase 1: Transfer Learning (Base frozen)\n")
start_time = time.time()

EPOCHS_PHASE1 = 20

history_phase1 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE1,
    validation_data=validation_generator,
    callbacks=[checkpoint, reduce_lr, early_stop],
    verbose=1
)

phase1_time = time.time() - start_time
print(f"\n✅ Phase 1 complete in {phase1_time/60:.1f} minutes")

## 🔓 Step 8: Fine-Tuning (Phase 2)

In [ ]:
# Unfreeze the last layers of base model for fine-tuning
base_model.trainable = True

# Freeze all layers except the last 30
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# New checkpoint for phase 2
checkpoint_phase2 = ModelCheckpoint(
    'ccmt_model_final.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

print("\n🚀 Starting Phase 2: Fine-Tuning (Last 30 layers unfrozen)\n")
start_time = time.time()

EPOCHS_PHASE2 = 15

history_phase2 = model.fit(
    train_generator,
    epochs=EPOCHS_PHASE2,
    validation_data=validation_generator,
    callbacks=[checkpoint_phase2, reduce_lr, early_stop],
    verbose=1
)

phase2_time = time.time() - start_time
total_training_time = phase1_time + phase2_time

print(f"\n✅ Phase 2 complete in {phase2_time/60:.1f} minutes")
print(f"\n🎯 Total training time: {total_training_time/60:.1f} minutes")

## 📊 Step 9: Training Visualizations

In [ ]:
# Combine histories from both phases
history_combined = {
    'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
    'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
    'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
    'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss']
}

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('🎯 Training Performance - CCMT CropGuard Model', fontsize=18, fontweight='bold', y=1.02)

# Accuracy plot
epochs_range = range(1, len(history_combined['accuracy']) + 1)
ax1.plot(epochs_range, history_combined['accuracy'], 'o-', color='#2ecc71', linewidth=2, label='Training Accuracy', markersize=4)
ax1.plot(epochs_range, history_combined['val_accuracy'], 's-', color='#3498db', linewidth=2, label='Validation Accuracy', markersize=4)
ax1.axvline(x=EPOCHS_PHASE1, color='red', linestyle='--', linewidth=2, label='Fine-Tuning Start', alpha=0.7)
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('📈 Model Accuracy', fontsize=14, fontweight='bold')
ax1.legend(loc='lower right', fontsize=10)
ax1.grid(True, alpha=0.3, linestyle='--')
ax1.set_ylim([0, 1])

# Loss plot
ax2.plot(epochs_range, history_combined['loss'], 'o-', color='#e74c3c', linewidth=2, label='Training Loss', markersize=4)
ax2.plot(epochs_range, history_combined['val_loss'], 's-', color='#f39c12', linewidth=2, label='Validation Loss', markersize=4)
ax2.axvline(x=EPOCHS_PHASE1, color='red', linestyle='--', linewidth=2, label='Fine-Tuning Start', alpha=0.7)
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax2.set_title('📉 Model Loss', fontsize=14, fontweight='bold')
ax2.legend(loc='upper right', fontsize=10)
ax2.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Print final metrics
final_train_acc = history_combined['accuracy'][-1]
final_val_acc = history_combined['val_accuracy'][-1]
final_train_loss = history_combined['loss'][-1]
final_val_loss = history_combined['val_loss'][-1]

print("\n" + "="*60)
print("📊 FINAL TRAINING RESULTS")
print("="*60)
print(f"✅ Training Accuracy: {final_train_acc*100:.2f}%")
print(f"✅ Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"📉 Training Loss: {final_train_loss:.4f}")
print(f"📉 Validation Loss: {final_val_loss:.4f}")
print(f"⏱️ Total Training Time: {total_training_time/60:.1f} minutes")
print("="*60)

## 🎯 Step 10: Evaluate Model & Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get predictions
print("\n🔍 Generating predictions on validation set...")
validation_generator.reset()
predictions = model.predict(validation_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = validation_generator.classes

# Classification report
print("\n" + "="*60)
print("📋 CLASSIFICATION REPORT")
print("="*60)
print(classification_report(true_classes, predicted_classes, target_names=class_names, digits=4))

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(18, 16))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='YlOrRd',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'},
    linewidths=0.5,
    linecolor='gray'
)
plt.title('🔥 Confusion Matrix - CCMT CropGuard Model', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Predicted Label', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Confusion matrix saved!")

## 📊 Step 11: Per-Class Accuracy Visualization

In [ ]:
# Calculate per-class accuracy
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
class_accuracy_dict = dict(zip(class_names, per_class_accuracy))

# Sort by accuracy
sorted_classes = sorted(class_accuracy_dict.items(), key=lambda x: x[1], reverse=True)
sorted_names = [x[0] for x in sorted_classes]
sorted_accuracies = [x[1]*100 for x in sorted_classes]

# Color code by accuracy
colors = ['#2ecc71' if acc >= 90 else '#f39c12' if acc >= 75 else '#e74c3c' for acc in sorted_accuracies]

# Plot per-class accuracy
plt.figure(figsize=(16, 10))
bars = plt.barh(sorted_names, sorted_accuracies, color=colors, edgecolor='black', linewidth=1.2, alpha=0.85)
plt.xlabel('Accuracy (%)', fontsize=14, fontweight='bold')
plt.ylabel('Disease Class', fontsize=14, fontweight='bold')
plt.title('🎯 Per-Class Accuracy - CCMT CropGuard Model', fontsize=18, fontweight='bold', pad=20)
plt.xlim([0, 105])
plt.grid(axis='x', alpha=0.3, linestyle='--')

# Add value labels on bars
for i, (bar, acc) in enumerate(zip(bars, sorted_accuracies)):
    plt.text(acc + 1, i, f'{acc:.1f}%', va='center', fontsize=10, fontweight='bold')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ecc71', label='Excellent (≥90%)'),
    Patch(facecolor='#f39c12', label='Good (75-89%)'),
    Patch(facecolor='#e74c3c', label='Needs Improvement (<75%)')
]
plt.legend(handles=legend_elements, loc='lower right', fontsize=11)

plt.tight_layout()
plt.savefig('per_class_accuracy.png', dpi=300, bbox_inches='tight')
plt.show()

# Print worst performing classes
print("\n⚠️ Classes needing improvement (<80% accuracy):")
for name, acc in sorted_classes:
    if acc < 0.8:
        print(f"   {name}: {acc*100:.1f}%")

## 💾 Step 12: Save Model & Export

In [ ]:
import json

# Save final model in multiple formats
print("\n💾 Saving model in multiple formats...\n")

# 1. HDF5 format (Keras)
model.save('ccmt_cropguard_final.h5')
print("✅ Saved: ccmt_cropguard_final.h5")

# 2. SavedModel format (TensorFlow)
model.save('ccmt_cropguard_savedmodel', save_format='tf')
print("✅ Saved: ccmt_cropguard_savedmodel/")

# 3. TFLite format (for mobile/edge)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('ccmt_cropguard_model.tflite', 'wb') as f:
    f.write(tflite_model)
print("✅ Saved: ccmt_cropguard_model.tflite (optimized)")

# Save class names
class_info = {
    'num_classes': num_classes,
    'class_names': class_names,
    'class_indices': train_generator.class_indices
}
with open('class_info.json', 'w') as f:
    json.dump(class_info, f, indent=2)
print("✅ Saved: class_info.json")

# Check file sizes
print("\n📦 Model File Sizes:")
!ls -lh ccmt_cropguard_final.h5
!ls -lh ccmt_cropguard_model.tflite

# Calculate model size
import os
h5_size = os.path.getsize('ccmt_cropguard_final.h5') / (1024*1024)
tflite_size = os.path.getsize('ccmt_cropguard_model.tflite') / (1024*1024)

print(f"\n📊 Model Sizes:")
print(f"   H5 Format: {h5_size:.2f} MB")
print(f"   TFLite Format: {tflite_size:.2f} MB (optimized)")
print(f"   Size Reduction: {(1 - tflite_size/h5_size)*100:.1f}%")

## 🎉 Step 13: Final Summary & Model Card

In [ ]:
# Create beautiful summary
print("\n" + "="*70)
print("🌱 CCMT CROPGUARD MODEL - TRAINING COMPLETE 🌱".center(70))
print("="*70)

print("\n📊 DATASET INFORMATION:")
print(f"   Total Images: {total_images:,}")
print(f"   Training Samples: {train_generator.samples:,}")
print(f"   Validation Samples: {validation_generator.samples:,}")
print(f"   Number of Classes: {num_classes}")
print(f"   Crops: Cashew, Cassava, Maize, Tomato")

print("\n🧠 MODEL ARCHITECTURE:")
print(f"   Base Model: MobileNetV2 (ImageNet pretrained)")
print(f"   Input Size: {IMG_SIZE}x{IMG_SIZE}x3")
print(f"   Total Parameters: {model.count_params():,}")
print(f"   Model Size (H5): {h5_size:.2f} MB")
print(f"   Model Size (TFLite): {tflite_size:.2f} MB")

print("\n🎯 PERFORMANCE METRICS:")
print(f"   Final Training Accuracy: {final_train_acc*100:.2f}%")
print(f"   Final Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"   Training Loss: {final_train_loss:.4f}")
print(f"   Validation Loss: {final_val_loss:.4f}")
print(f"   Total Epochs: {len(history_combined['accuracy'])}")
print(f"   Training Time: {total_training_time/60:.1f} minutes")

print("\n📁 SAVED FILES:")
print("   ✅ ccmt_cropguard_final.h5 - Full Keras model")
print("   ✅ ccmt_cropguard_model.tflite - Optimized for mobile/edge")
print("   ✅ ccmt_cropguard_savedmodel/ - TensorFlow SavedModel format")
print("   ✅ class_info.json - Class names and indices")
print("   ✅ training_history.png - Training curves")
print("   ✅ confusion_matrix.png - Confusion matrix heatmap")
print("   ✅ per_class_accuracy.png - Per-class performance")

print("\n🚀 NEXT STEPS:")
print("   1. Download all model files to your computer")
print("   2. Upload ccmt_cropguard_final.h5 to Google Drive")
print("   3. Update your CropGuard app to load this model")
print("   4. Test with real crop images")
print("   5. Deploy to Streamlit Cloud")

print("\n" + "="*70)
print("🎉 CONGRATULATIONS! Your model is ready to use! 🎉".center(70))
print("="*70 + "\n")

## 📥 Step 14: Download Files to Your Computer

In [ ]:
# If you need to download files from Colab
from google.colab import files

print("📥 Preparing files for download...\n")

# Download main model file
print("Downloading: ccmt_cropguard_final.h5")
files.download('ccmt_cropguard_final.h5')

# Download TFLite model
print("Downloading: ccmt_cropguard_model.tflite")
files.download('ccmt_cropguard_model.tflite')

# Download class info
print("Downloading: class_info.json")
files.download('class_info.json')

# Download visualizations
print("Downloading: training_history.png")
files.download('training_history.png')

print("Downloading: confusion_matrix.png")
files.download('confusion_matrix.png')

print("Downloading: per_class_accuracy.png")
files.download('per_class_accuracy.png')

print("\n✅ All files downloaded!")